In [1]:
import json
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import tensorflow as tf
import os


# Caricamento dei dati dal file JSON
with open('../RemovedStopWordData/ate_absita_training_extracted.json', 'r') as file:
    data_train = json.load(file)

with open('../RemovedStopWordData/ate_absita_gold_extracted.json', 'r') as file:
    data_test = json.load(file)

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-uncased")
model = TFBertForSequenceClassification.from_pretrained("dbmdz/bert-base-italian-xxl-uncased")

train_sentences = data_train['aspects']
train_labels = data_train['polarities']



train_sentences_new = []
train_labels_new = []
for index, (aspects, labels) in enumerate(zip(train_sentences,train_labels)):
    for jndex, (aspect, label) in enumerate(zip(aspects,labels)):
        train_sentences_new.append(aspect)
        train_labels_new.append(label)

print(len(train_sentences_new))
print(len(train_labels_new))
    

# Tokenizzazione dei dati di addestramento
train_encodings = tokenizer.batch_encode_plus(train_sentences_new, truncation=True, padding=True, max_length=128, return_tensors="tf")

# Creazione del dataset TensorFlow per l'addestramento
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': tf.convert_to_tensor(train_encodings['input_ids']),
        'attention_mask': tf.convert_to_tensor(train_encodings['attention_mask'])
    },
    tf.convert_to_tensor(train_labels_new)
))

# Addestramento del modello
model.compile(optimizer="adam", loss="categorical_crossentropy")
model.fit(train_dataset.shuffle(1000).batch(16), epochs=4)

test_aspect_texts = data_test['aspects']
test_labels = data_test['polarities']

train_sentences_new = []
train_labels_new = []
for index, (aspects, labels) in enumerate(zip(test_aspect_texts,test_labels)):
    for jndex, (aspect, label) in enumerate(zip(aspects,labels)):
        train_sentences_new.append(aspect)
        train_labels_new.append(label)

# Esempio di frasi di input per le predizioni
#input_sentences = ["Il design è molto elegante.", "Il prodotto non funziona correttamente."]

# Tokenizzazione dei dati di input per le predizioni
input_encodings = tokenizer.batch_encode_plus(train_sentences_new, truncation=True, padding=True, max_length=128, return_tensors="tf")

# Creazione del dataset TensorFlow per le predizioni
input_dataset = tf.data.Dataset.from_tensor_slices(dict(input_encodings)).batch(16)

# Predizioni
predictions = model.predict(input_dataset)



print(predictions)


c:\Users\Ivan\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Ivan\miniconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ivan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: http

AssertionError: Size mismatch between tensors